# Métricas
---
Las metricas utilizadas en *Forescast data driven models* son diferentes a las utilizadas en otras áreas de *Deep learning*.

## latitude-weighted Root Mean Square Error (RMSE)
---
Se calculas para cada varaible para un nivel verical dado (i.e. $T500$ temperatura a 500 m de altura $RMSE_{500}$).

Es el RMSE pero promediado a traves de las dimensiones:
> * Longitud $j$: $[1, \ldots, J]$ 
> * Latitud: $i$: $[1, \ldots, I]$
> * Tiempo: $t$: $[1, \ldots, T]$

$$

\mathrm{RMSE}_l=\sqrt{\frac{1}{T I J} \sum_t^T \sum_i^I \sum_j^J w(i)\left(f_{t, i, j}-o_{t, i, j}\right)^2}

$$

> **Donde:**
> * $f$: Forecast 
> * $o$: Ground truth (ERA 5) 
> * $w$: Latitud weights 

**Latitud weights:** Debido a que las celdas en altas latitudes son mas pequeñas comparadas con las de bajas latitudes se pondera el error de la siguiente forma:

$$

w(i)=\frac{\sin \theta_i^{\mathrm{u}}-\sin \theta_i^{\mathrm{l}}}{\frac{1}{I} \sum_i^I\left(\sin \theta_i^{\mathrm{u}}-\sin \theta_i^{\mathrm{l}}\right)}

$$

Donde $θ^u_i$ and $θ^l_i$ indican los límites superior e inferior de la celda con indice de latitud $i$ respectivamente.